In [ ]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
from pypardiso import spsolve
from pathlib import Path
from gsa_framework.utils import read_pickle, write_pickle
from gsa_framework.visualization.plotting import plot_histogram_Y1_Y2
from fs.zipfs import ZipFS
from copy import deepcopy
from consumption_model_ch.utils import get_habe_filepath
import pandas as pd
import sys
sys.path.append('/Users/akim/PycharmProjects/akula')
from collections import Counter

from akula.sensitivity_analysis import get_mask

project = 'GSA for archetypes'
bd.projects.set_current(project)
iterations = 2000
seed = 11111000 

fp_paper_2 = Path("write_files") / project.lower().replace(" ", "_") / "paper_2"
fp_paper_2.mkdir(exist_ok=True, parents=True)

In [ ]:
co = bd.Database('swiss consumption 1.0')
fu = [act for act in co if "Food" in act['name']][0]
demand = {fu: 1}
method = ("IPCC 2013", "climate change", "GWP 100a", "uncertain")
fu_mapped, packages, _ = bd.prepare_lca_inputs(demand=demand, method=method, remapping=False)
lca = bc.LCA(demand=fu_mapped, data_objs=packages, use_distributions=False)
lca.lci()
lca.lcia()
lca.score

In [ ]:
bs = bd.Database('biosphere3').datapackage()
ei = bd.Database('ecoinvent 3.8 cutoff').datapackage()
co = bd.Database('swiss consumption 1.0').datapackage()
me = bd.Method(method).datapackage()

co_indices = co.get_resource("swiss_consumption_1.0_technosphere_matrix.indices")[0]
co_flip = co.get_resource("swiss_consumption_1.0_technosphere_matrix.flip")[0]
# co_vector = co.get_resource("swiss_consumption_1.0_technosphere_matrix.data")[0]
# co_data = np.random.rand(len(co_vector), iterations)
# get_co_data()

In [ ]:
def get_household_data(indices, co_name="swiss consumption 1.0"):
    # 1. Get some metadata from the consumption database
    co = bd.Database(co_name)
    year_habe = co.metadata['year_habe']
    dir_habe = co.metadata['dir_habe']

    # 2. Extract total demand from HABE
    path_ausgaben = get_habe_filepath(dir_habe, year_habe, 'Ausgaben')
    path_mengen = get_habe_filepath(dir_habe, year_habe, 'Mengen')
    path_konsumgueter = get_habe_filepath(dir_habe, year_habe, 'Konsumgueter')

    # change codes to be consistent with consumption database and Andi's codes
    ausgaben = pd.read_csv(path_ausgaben, sep='\t')
    mengen = pd.read_csv(path_mengen, sep='\t')
    konsumgueter = pd.read_csv(path_konsumgueter, sep='\t')
    ausgaben.columns = [col.lower() for col in ausgaben.columns]
    mengen.columns = [col.lower() for col in mengen.columns]
    konsumgueter.columns = [col.lower() for col in konsumgueter.columns]
    codes_co_db = sorted([act['code'] for act in co])
    columns_a = ausgaben.columns.values
    columns_m = [columns_a[0]]
    columns_k = [columns_a[0]]
    codes_m = []
    for code_a in columns_a[1:]:
        code_m = code_a.replace('a', 'm')
        if code_m in codes_co_db:
            columns_m.append(code_m)
            codes_m.append(code_m)
        else:
            columns_m.append(code_a)
    ausgaben.columns = columns_m
    # Replace ausgaben data with mengen data
    for code_m in codes_m:
        ausgaben[code_m] = mengen[code_m].values

    data = np.zeros((0,len(ausgaben)))
    for inds in indices:
        input_code = bd.get_activity(inds[0])['code']
        try:
            data_current = ausgaben[input_code].values
            data = np.vstack([data, data_current])
        except:
            print(input_code)

    return data

In [ ]:
# Foreground uncertainty weighted by number of people per household
def get_household_data_weighted(indices, co_name="swiss consumption 1.0"):
    # 1. Get some metadata from the consumption database
    co = bd.Database(co_name)
    year_habe = co.metadata['year_habe']
    dir_habe = co.metadata['dir_habe']

    # 2. Extract total demand from HABE
    path_ausgaben = get_habe_filepath(dir_habe, year_habe, 'Ausgaben')
    path_mengen = get_habe_filepath(dir_habe, year_habe, 'Mengen')
    path_konsumgueter = get_habe_filepath(dir_habe, year_habe, 'Konsumgueter')
    path_personen = get_habe_filepath(dir_habe, year_habe, 'Personen')

    # change codes to be consistent with consumption database and Andi's codes
    ausgaben = pd.read_csv(path_ausgaben, sep='\t')
    mengen = pd.read_csv(path_mengen, sep='\t')
    konsumgueter = pd.read_csv(path_konsumgueter, sep='\t')
    personen_raw = pd.read_csv(path_personen, sep='\t')
    ausgaben.columns = [col.lower() for col in ausgaben.columns]
    mengen.columns = [col.lower() for col in mengen.columns]
    konsumgueter.columns = [col.lower() for col in konsumgueter.columns]
    personen_raw.columns = [col.lower() for col in personen_raw.columns]
    num_personen = dict(Counter(personen_raw["haushaltid"]))
    num_personen = [{'haushaltid': k, "n_personen": v} for k,v in num_personen.items()]
    personen = pd.DataFrame.from_dict(num_personen)

    codes_co_db = sorted([act['code'] for act in co])
    columns_a = ausgaben.columns.values
    columns_m = [columns_a[0]]
    columns_k = [columns_a[0]]
    codes_m = []
    for code_a in columns_a[1:]:
        code_m = code_a.replace('a', 'm')
        if code_m in codes_co_db:
            columns_m.append(code_m)
            codes_m.append(code_m)
        else:
            columns_m.append(code_a)
    ausgaben.columns = columns_m
    # Replace ausgaben data with mengen data
    for code_m in codes_m:
        ausgaben[code_m] = mengen[code_m].values
    weighted_ausgaben = pd.concat(
        [
            personen.set_index('haushaltid'),
            ausgaben.set_index('haushaltid'), 
        ],
        join='inner',
        axis=1
    )
    weighted_ausgaben = weighted_ausgaben.iloc[:,1:].div(weighted_ausgaben['n_personen'], axis=0)
    weighted_ausgaben = weighted_ausgaben.reset_index()

    data = np.zeros((0,len(weighted_ausgaben)))
    for inds in indices:
        input_code = bd.get_activity(inds[0])['code']
        try:
            data_current = weighted_ausgaben[input_code].values
            data = np.vstack([data, data_current])
        except:
            print(input_code)

    return data

In [ ]:
use_indices = [(exc.input.id, fu.id) for exc in fu.exchanges() if exc['type']!='production']
use_mask = get_mask(co_indices, use_indices)
use_flip = co_flip[use_mask]

household_data = get_household_data(use_indices)
choice = np.sort(np.random.choice(household_data.shape[1], iterations, replace=True))
use_data = household_data[:, choice]

household_data_weighted = get_household_data_weighted(use_indices)
choice_weighted = np.sort(np.random.choice(household_data_weighted.shape[1], iterations, replace=True))
use_data_weighted = household_data_weighted[:, choice]

use_indices = np.array(use_indices, dtype=[('row', '<i4'), ('col', '<i4')])

In [ ]:
ppl_per_hh = 2.2415871421396285

In [ ]:
# co_static = co.exclude({"matrix": 'technosphere_matrix'})
co_static = co
co_uncertain = bwp.create_datapackage(sequential=True)
co_uncertain.add_persistent_array(
    matrix="technosphere_matrix",
    indices_array=use_indices,
    name="swiss_consumption_1.0_technosphere_matrix",
    data_array=use_data,
    flip_array=use_flip,
)
co_uncertain_weighted = bwp.create_datapackage(sequential=True)
co_uncertain_weighted.add_persistent_array(
    matrix="technosphere_matrix",
    indices_array=use_indices,
    name="swiss_consumption_1.0_weighted_technosphere_matrix",
    data_array=use_data_weighted,
    flip_array=use_flip,
)

In [ ]:
me_static = me.exclude({"kind": "distributions"})
ei_static = ei.exclude({"kind": "distributions"})
bs_static = bs.exclude({"kind": "distributions"})

In [ ]:
dps_bg = [me, bs, ei, co_static]
dps_bg_fg = [me, bs, ei, co_static, co_uncertain]
dps_fg = [me, bs, ei_static, co_static, co_uncertain]
dps_bg_fg_weighted = [me, bs, ei, co_static, co_uncertain_weighted]

In [ ]:
fp_monte_carlo_bg_fg = fp_paper_2 / "{}.{}.{}.pickle".format(
    "bg+fg", iterations, seed
)
fp_monte_carlo_bg_fg_weighted = fp_paper_2 / "{}.{}.{}.pickle".format(
    "bg+fg+weighted", iterations, seed
)
fp_monte_carlo_bg = fp_paper_2 / "{}.{}.{}.pickle".format(
    "bg", iterations, seed
)
fp_monte_carlo_fg = fp_paper_2 / "{}.{}.{}.pickle".format(
    "fg", iterations, seed
)

options = {
    "bg+fg": {
        "fp": fp_monte_carlo_bg_fg,
        "dps": dps_bg_fg,
    },
    "bg": {
        "fp": fp_monte_carlo_bg,
        "dps": dps_bg,
    },
    "bg+fg+weighted": {
        "fp": fp_monte_carlo_bg_fg_weighted,
        "dps": dps_bg_fg_weighted,
    },
#     "fg": {
#         "fp": fp_monte_carlo_fg,
#         "dps": dps_fg,
#     }
} 

In [ ]:
%%time
scores = {}
for option, data in options.items():
    print(option)
    fp = data['fp']
    dps = data['dps']
    if fp.exists():
        scores[option] = read_pickle(fp)
    else:
        if option=='fg':
            use_distributions = False
        else:
            use_distributions = True
        dict_for_lca = dict(
            use_distributions=use_distributions,
            use_arrays=True,
            seed_override=seed,
        )
        lca_new = bc.LCA(
            fu_mapped,
            data_objs=dps,
            **dict_for_lca,
        )
        lca_new.lci()
        lca_new.lcia()
        scores_current = []
        for i in range(iterations):
            next(lca_new)
            scores_current.append(lca_new.score)
        scores[option] = scores_current
        write_pickle(scores[option], fp)
    

In [ ]:
scores['bg+fg+weighted']

In [ ]:
import plotly.graph_objects as go
start = 0
end = 200
fig = go.Figure()
for option, data in scores.items():
    data = np.array(data)
    if option == 'bg':
        data /= ppl_per_hh
    fig.add_trace(
        go.Scatter(
            x=np.arange(iterations),
            y=data[start:end],
            name=option,
            mode="lines+markers",
            showlegend=True,
        ),
    )
    
fig.show()

In [ ]:
min(scores['bg+fg']), max(scores['bg+fg'])

In [ ]:
bin_min = 0
bin_max = 1000#max(scores['bg+fg'])
num_bins = 150
opacity = 0.65

bins_ = np.linspace(bin_min, bin_max, num_bins, endpoint=True)
color_bg = "rgb(29,105,150)"
color_bg_fg = "rgb(148, 52, 110)" #"rgb(36, 121, 108)"
color_gray_hex = "#b2bcc0"
color_darkgray_hex = "#485063"
color_black_hex = "#212931"
lca_scores_axis_title = r"$\text{LCIA scores, [kg CO}_2\text{-eq}]$"

fig = go.Figure()

# Background + foreground
freq1, bins1 = np.histogram(scores['bg+fg+weighted'], bins=bins_)
fig.add_trace(
    go.Scatter(
        x=bins1,
        y=freq1,
        name=r"$\text{Background and foreground vary}$",
        opacity=opacity,
        line=dict(color=color_bg_fg, width=1, shape="hvh"),
        showlegend=True,
        fill="tozeroy",
    ),
)

# Background
freq2, bins2 = np.histogram(np.array(scores['bg'])/ppl_per_hh, bins=bins_)
fig.add_trace(
    go.Scatter(
        x=bins2,
        y=freq2,
        name=r"$\text{Only background varies}$",
        opacity=opacity,
        line=dict(color=color_bg, width=1, shape="hvh"),
        showlegend=True,
        fill="tozeroy",
    ),
)

fig.update_xaxes(
    title_text=lca_scores_axis_title,
    showgrid=True,
    gridwidth=1,
    gridcolor=color_gray_hex,
    zeroline=True,
    zerolinewidth=1,
    zerolinecolor=color_gray_hex,
    showline=True,
    linewidth=1,
    linecolor=color_gray_hex,
)
fig.update_yaxes(
    title_text=r"$\text{Frequency}$", 
    range=[-10,550],
    showgrid=True,
    gridwidth=1,
    gridcolor=color_gray_hex,
    zeroline=True,
    zerolinewidth=1,
    zerolinecolor=color_black_hex,
    showline=True,
    linewidth=1,
    linecolor=color_gray_hex,
)
fig.update_layout(
    width=600,
    height=250,
    paper_bgcolor="rgba(255,255,255,1)",
    plot_bgcolor="rgba(255,255,255,1)",
    legend=dict(
        x=0.7,
        y=0.90,
        orientation="v",
        xanchor="center",
        font=dict(size=12),
        # bgcolor=color_lightgray_hex,
        bordercolor=color_darkgray_hex,
        borderwidth=1,
    ),
    margin=dict(l=65, r=0, t=0, b=0),
)

# filepath_fig = fp_paper_2 / "lca_scores_uncertainty_bg_fg.pdf"
# fig.write_image(filepath_fig.as_posix())

In [ ]:
# filepath_fig = Path("lca_scores_uncertainty_bg_fg_per_capita_impacts.pdf")
# fig.write_image(filepath_fig.as_posix())